In [37]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.feature_selection import RFECV

from sklearn.preprocessing import scale
from sklearn.decomposition import PCA

In [38]:
user = pd.read_csv("analysis_data.csv", sep=",", index_col="Client")
print(user.shape)
user.head()

(1581, 41)


,Age,Tenure,M,Count_CA,Count_SA,Count_MF,Count_OVD,Count_CC,Count_CL,ActBal_CA,...,Sale_CL,Revenue_MF,Revenue_CC,Revenue_CL,Actbal_CA_avg,ActBal_SA_avg,ActBal_MF_avg,ActBal_OVD_avg,ActBal_CC_avg,ActBal_CL_avg
Client,,,,,,,,,,,,,,,,,,,,,
909,21,27,1,1,0.0,0.0,1.0,0.0,1.0,4.710714,...,NaN,NaN,NaN,NaN,4.710714,0.00000,0.0,0.0,0.00,4291.996429
1217,38,165,1,1,0.0,0.0,0.0,0.0,0.0,6752.244643,...,0.0,0.000000,0.000000,0.0,6752.244643,0.00000,0.0,0.0,0.00,0.000000
850,49,44,0,1,0.0,0.0,0.0,0.0,0.0,43.523214,...,0.0,0.000000,0.893929,0.0,43.523214,0.00000,0.0,0.0,0.00,0.000000
1473,54,34,1,1,1.0,0.0,0.0,1.0,1.0,29.024286,...,0.0,4.929643,0.000000,0.0,29.024286,14447.80143,0.0,0.0,653.91,1132.590357
1038,29,106,1,1,0.0,0.0,0.0,0.0,0.0,27.035714,...,0.0,0.000000,0.000000,0.0,27.035714,0.00000,0.0,0.0,0.00,0.000000


In [39]:
user_tr = user[pd.notnull(user['Sale_MF'])]
print(user_tr.shape)
user_tr.head()

(947, 41)


,Age,Tenure,M,Count_CA,Count_SA,Count_MF,Count_OVD,Count_CC,Count_CL,ActBal_CA,...,Sale_CL,Revenue_MF,Revenue_CC,Revenue_CL,Actbal_CA_avg,ActBal_SA_avg,ActBal_MF_avg,ActBal_OVD_avg,ActBal_CC_avg,ActBal_CL_avg
Client,,,,,,,,,,,,,,,,,,,,,
1217,38,165,1,1,0.0,0.0,0.0,0.0,0.0,6752.244643,...,0.0,0.000000,0.000000,0.000000,6752.244643,0.00000,0.0,0.0000,0.00,0.000000
850,49,44,0,1,0.0,0.0,0.0,0.0,0.0,43.523214,...,0.0,0.000000,0.893929,0.000000,43.523214,0.00000,0.0,0.0000,0.00,0.000000
1473,54,34,1,1,1.0,0.0,0.0,1.0,1.0,29.024286,...,0.0,4.929643,0.000000,0.000000,29.024286,14447.80143,0.0,0.0000,653.91,1132.590357
1038,29,106,1,1,0.0,0.0,0.0,0.0,0.0,27.035714,...,0.0,0.000000,0.000000,0.000000,27.035714,0.00000,0.0,0.0000,0.00,0.000000
225,14,187,1,1,0.0,0.0,1.0,0.0,0.0,345.686071,...,1.0,0.000000,0.000000,8.130714,345.686071,0.00000,0.0,618.3975,0.00,0.000000


In [40]:
user_tr = user_tr.drop(['Sale_CC','Sale_CL','Revenue_MF','Revenue_CC','Revenue_CL'], axis=1)
user_tr.head()

,Age,Tenure,M,Count_CA,Count_SA,Count_MF,Count_OVD,Count_CC,Count_CL,ActBal_CA,...,TransactionsDebCash_Card,TransactionsDebCashless_Card,TransactionsDeb_PaymentOrder,Sale_MF,Actbal_CA_avg,ActBal_SA_avg,ActBal_MF_avg,ActBal_OVD_avg,ActBal_CC_avg,ActBal_CL_avg
Client,,,,,,,,,,,,,,,,,,,,,
1217,38,165,1,1,0.0,0.0,0.0,0.0,0.0,6752.244643,...,0,0,1,0.0,6752.244643,0.00000,0.0,0.0000,0.00,0.000000
850,49,44,0,1,0.0,0.0,0.0,0.0,0.0,43.523214,...,0,0,1,0.0,43.523214,0.00000,0.0,0.0000,0.00,0.000000
1473,54,34,1,1,1.0,0.0,0.0,1.0,1.0,29.024286,...,1,26,11,1.0,29.024286,14447.80143,0.0,0.0000,653.91,1132.590357
1038,29,106,1,1,0.0,0.0,0.0,0.0,0.0,27.035714,...,0,0,0,0.0,27.035714,0.00000,0.0,0.0000,0.00,0.000000
225,14,187,1,1,0.0,0.0,1.0,0.0,0.0,345.686071,...,0,4,1,0.0,345.686071,0.00000,0.0,618.3975,0.00,0.000000


In [41]:
print(user_tr.shape)

(947, 36)


In [42]:
user_tr_X = user_tr.drop('Sale_MF', axis=1)
user_tr_Y = user_tr['Sale_MF']

In [43]:
X_train, X_test, y_train, y_test = train_test_split(user_tr_X, user_tr_Y, test_size=0.2, random_state=42)

In [44]:
logreg = LogisticRegression()
logreg.fit(X_train,y_train)


y_pred = logreg.predict(X_test)
print(roc_auc_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

y_pred_train = logreg.predict(X_train)
print(roc_auc_score(y_train, y_pred_train))
print(classification_report(y_train, y_pred_train))


0.515714285714
             precision    recall  f1-score   support

        0.0       0.74      0.97      0.84       140
        1.0       0.43      0.06      0.11        50

avg / total       0.66      0.73      0.65       190

0.616190542711
             precision    recall  f1-score   support

        0.0       0.85      0.98      0.91       618
        1.0       0.74      0.25      0.38       139

avg / total       0.83      0.85      0.81       757



In [45]:
logreg = LogisticRegression()
scores = cross_val_score(logreg, user_tr_X, user_tr_Y, cv=10, scoring='roc_auc', n_jobs=-1)
print("roc_auc: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

roc_auc: 0.63 (+/- 0.17)


In [46]:
svc = SVC(kernel='rbf', degree=2, random_state=101)
scores = cross_val_score(svc, user_tr_X, user_tr_Y, cv=10, scoring='roc_auc', n_jobs=-1)
print("roc_auc: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

roc_auc: 0.50 (+/- 0.01)


In [47]:
random_forest = RandomForestClassifier(n_estimators = 25)
scores = cross_val_score(random_forest, user_tr_X, user_tr_Y, cv=10, scoring='roc_auc', n_jobs=-1)
print("roc_auc: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

roc_auc: 0.54 (+/- 0.11)


In [48]:
gaussian = GaussianNB()
scores = cross_val_score(gaussian, user_tr_X, user_tr_Y, cv=10, scoring='roc_auc', n_jobs=-1)
print("roc_auc: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

roc_auc: 0.57 (+/- 0.13)


In [49]:
#Scale the 
user_tr_X_scale = scale(user_tr_X)
print(user_tr_X_scale)

[[-0.26748029  0.98414783  0.93952316 ..., -0.21893948 -0.20892838
  -0.21074605]
 [ 0.32422542 -0.85655318 -1.06436971 ..., -0.21893948 -0.20892838
  -0.21074605]
 [ 0.59318256 -1.00867723  0.93952316 ..., -0.21893948  3.97811872
   0.57597595]
 ..., 
 [-1.07435172  0.9080858   0.93952316 ..., -0.21893948 -0.20892838
  -0.21074605]
 [ 0.16285114 -0.6283671   0.93952316 ..., -0.21893948 -0.20892838
  -0.21074605]
 [ 0.96972256  1.10584707 -1.06436971 ..., -0.21893948 -0.20892838
  -0.21074605]]


In [50]:
logreg = LogisticRegression()
scores = cross_val_score(logreg, user_tr_X_scale, user_tr_Y, cv=10, scoring='roc_auc', n_jobs=-1)
print("roc_auc: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

roc_auc: 0.63 (+/- 0.18)


In [51]:
#with penalty
logreg = LogisticRegression(penalty='l1')
logreg.fit(X_train,y_train)
scores = cross_val_score(logreg, user_tr_X_scale, user_tr_Y, cv=10, scoring='roc_auc', n_jobs=-1)
print("roc_auc: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print(scores)


roc_auc: 0.63 (+/- 0.18)
[ 0.7534626   0.70152355  0.52770083  0.66689751  0.67590028  0.49792244
  0.64404432  0.54362881  0.72350877  0.53407407]


In [52]:
logreg = LogisticRegression(penalty='l2')
selector = RFECV(estimator=logreg, step=1, cv=10,scoring='roc_auc')

selector.fit(user_tr_X_scale, user_tr_Y)
print("Optimal number of features : %d" % selector.n_features_)

Optimal number of features : 19


In [53]:
user_tr_X_scale_transform = selector.transform(user_tr_X_scale)

In [54]:
search = {'penalty': ['l1','l2'],'C': [0.001, 0.1, 1, 10, 100, 1000], 'class_weight': ['balanced', None]}


logreg = LogisticRegression()

logreg_grid = GridSearchCV(estimator=logreg,param_grid=search, scoring='roc_auc', n_jobs=-1,refit=True, cv=10)
logreg_grid.fit(user_tr_X_scale_transform,user_tr_Y)
print(logreg_grid.best_estimator_)

print(logreg_grid.best_params_)

print(logreg_grid.best_score_)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
{'C': 0.1, 'class_weight': None, 'penalty': 'l2'}
0.652063765538


In [55]:
svc = SVC(kernel='rbf', degree=2, random_state=101)
scores = cross_val_score(svc, user_tr_X_scale, user_tr_Y, cv=10, scoring='roc_auc', n_jobs=-1)
print("roc_auc: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print(scores)

roc_auc: 0.59 (+/- 0.13)
[ 0.72091413  0.53601108  0.61218837  0.61911357  0.6398892   0.51385042
  0.57479224  0.62880886  0.6154386   0.47777778]


In [56]:
search = search_dict = {'kernel': ['linear','rbf'],'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001]}


svc = SVC()

svc_grid = RandomizedSearchCV(estimator=svc,param_distributions=search, scoring='roc_auc', n_jobs=-1,refit=True, cv=3)
svc_grid.fit(user_tr_X_scale,user_tr_Y)
print(svc_grid.best_estimator_)

print(svc_grid.best_params_)

print(svc_grid.best_score_)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
{'kernel': 'rbf', 'gamma': 0.0001, 'C': 1}
0.622855165943


In [57]:
svc = SVC(kernel='linear', degree=2, random_state=101)
scores = cross_val_score(svc, user_tr_X_scale, user_tr_Y, cv=10, scoring='roc_auc', n_jobs=-1)
print(scores)
print("roc_auc: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[ 0.67174515  0.70914127  0.51177285  0.64750693  0.66481994  0.49722992
  0.57825485  0.52493075  0.66315789  0.47703704]
roc_auc: 0.59 (+/- 0.16)


In [58]:
random_forest = RandomForestClassifier(n_estimators = 20)
scores = cross_val_score(random_forest, user_tr_X_scale, user_tr_Y, cv=10, scoring='roc_auc', n_jobs=-1)
print("roc_auc: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

roc_auc: 0.56 (+/- 0.19)


In [59]:
gaussian = GaussianNB()
scores = cross_val_score(gaussian, user_tr_X_scale, user_tr_Y, cv=10, scoring='roc_auc', n_jobs=-1)
print("roc_auc: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

roc_auc: 0.57 (+/- 0.12)


In [60]:
#dimension reduction
pca = PCA(n_components=20)
X_pca_20 = pca.fit_transform(user_tr_X_scale)
X_pca_20.shape

(947, 20)

In [61]:
svc = SVC(kernel='linear', degree=2, random_state=101)
scores = cross_val_score(svc, X_pca_20, user_tr_Y, cv=10, scoring='roc_auc', n_jobs=-1)
print(scores)
print("roc_auc: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[ 0.58171745  0.6267313   0.45706371  0.61288089  0.63642659  0.45290859
  0.52285319  0.45290859  0.42877193  0.4362963 ]
roc_auc: 0.52 (+/- 0.16)


In [62]:
#with penalty
logreg = LogisticRegression(penalty='l1', C=1)
scores = cross_val_score(logreg, X_pca_20, user_tr_Y, cv=20, scoring='roc_auc', n_jobs=-1)
print(scores)
print("roc_auc: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[ 0.60526316  0.63157895  0.75        0.53947368  0.46842105  0.53684211
  0.54736842  0.52631579  0.65789474  0.56725146  0.34795322  0.42690058
  0.54093567  0.69883041  0.73099415  0.3625731   0.61403509  0.62280702
  0.44444444  0.47147147]
roc_auc: 0.55 (+/- 0.22)


In [63]:
random_forest = RandomForestClassifier(n_estimators = 50)
scores = cross_val_score(random_forest, X_pca_20, user_tr_Y, cv=10, scoring='roc_auc', n_jobs=-1)
print("roc_auc: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

roc_auc: 0.56 (+/- 0.12)


In [64]:
svc_lin = LinearSVC()
scores = cross_val_score(svc_lin, X_pca_20, user_tr_Y, cv=10, scoring='roc_auc', n_jobs=-1)
print(scores)
print("roc_auc: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[ 0.62257618  0.63088643  0.52700831  0.63157895  0.55470914  0.41759003
  0.63157895  0.52908587  0.65263158  0.45481481]
roc_auc: 0.57 (+/- 0.16)


In [65]:
gaussian = GaussianNB()
scores = cross_val_score(gaussian, X_pca_20, user_tr_Y, cv=10, scoring='roc_auc', n_jobs=-1)
print("roc_auc: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

roc_auc: 0.52 (+/- 0.11)


In [66]:
search = {'penalty': ['l1','l2'],'C': [0.001, 0.1, 1, 10, 100, 1000], 'class_weight': ['balanced', None]}


logreg = LogisticRegression()

logreg_grid = GridSearchCV(estimator=logreg,param_grid=search, scoring='roc_auc', n_jobs=-1,refit=True, cv=10)
logreg_grid.fit(X_pca_20,user_tr_Y)
print(logreg_grid.best_estimator_)

print(logreg_grid.best_params_)

print(logreg_grid.best_score_)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
{'C': 0.1, 'class_weight': None, 'penalty': 'l2'}
0.563409227045
